In [48]:
#import libraries
import pandas as pd
import requests
from sqlalchemy import create_engine

In [49]:
#read in data
base_url = 'https://data.lacity.org/resource/2nrs-mtv8.json'
params = {
    '$limit': 1000  # Limit to 1000 records per request
}
total_records = 1000000
iterations = int(total_records/1000)

In [50]:
#set empty array for all data
all_records = []
for i in range(iterations):
    offset = i * 1000
    params['$offset'] = offset
    response = requests.get(base_url,params=params)
    data=response.json()
    all_records.extend(data)
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [53]:
#check data types for each column
df.dtypes

id                              int64
dt_reported            datetime64[ns]
dt_occurred            datetime64[ns]
time_occ                        int64
area                            int64
area_name                      object
district_code                   int64
part_1_2                        int64
crime_code                      int64
crime_description              object
vict_age                        int64
vict_sex                       object
vict_descent                   object
premise_code                    int64
premise_description            object
status                         object
status_desc                    object
crm_cd_1                        int64
crm_cd_2                        int64
location                       object
lat                           float64
lon                           float64
mocodes                        object
weapon_used_cd                  int64
weapon_desc                    object
cross_street                   object
crm_cd_3    

In [54]:
df = pd.DataFrame(all_records)
df.head()

,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,...,crm_cd_2,location,lat,lon,mocodes,weapon_used_cd,weapon_desc,cross_street,crm_cd_3,crm_cd_4
0,190326475,2020-03-01T00:00:00.000,2020-03-01T00:00:00.000,2130,07,Wilshire,0784,1,510,VEHICLE - STOLEN,...,998,1900 S LONGWOOD AV,34.0375,-118.3506,NaN,NaN,NaN,NaN,NaN,NaN
1,200106753,2020-02-09T00:00:00.000,2020-02-08T00:00:00.000,1800,01,Central,0182,1,330,BURGLARY FROM VEHICLE,...,998,1000 S FLOWER ST,34.0444,-118.2628,1822 1402 0344,NaN,NaN,NaN,NaN,NaN
2,200320258,2020-11-11T00:00:00.000,2020-11-04T00:00:00.000,1700,03,Southwest,0356,1,480,BIKE - STOLEN,...,NaN,1400 W 37TH ST,34.021,-118.3002,0344 1251,NaN,NaN,NaN,NaN,NaN
3,200907217,2023-05-10T00:00:00.000,2020-03-10T00:00:00.000,2037,09,Van Nuys,0964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,NaN,14000 RIVERSIDE DR,34.1576,-118.4387,0325 1501,NaN,NaN,NaN,NaN,NaN
4,220614831,2022-08-18T00:00:00.000,2020-08-17T00:00:00.000,1200,06,Hollywood,0666,2,354,THEFT OF IDENTITY,...,NaN,1900 TRANSIENT,34.0944,-118.3277,1822 1501 0930 2004,NaN,NaN,NaN,NaN,NaN


In [55]:
#check length of df
len(df)

944235

In [56]:
df.columns

Index(['dr_no', 'date_rptd', 'date_occ', 'time_occ', 'area', 'area_name',
       'rpt_dist_no', 'part_1_2', 'crm_cd', 'crm_cd_desc', 'vict_age',
       'vict_sex', 'vict_descent', 'premis_cd', 'premis_desc', 'status',
       'status_desc', 'crm_cd_1', 'crm_cd_2', 'location', 'lat', 'lon',
       'mocodes', 'weapon_used_cd', 'weapon_desc', 'cross_street', 'crm_cd_3',
       'crm_cd_4'],
      dtype='object')

In [57]:
#rename columns
df = df.rename(columns={'dr_no':'id', 'date_rptd':'dt_reported', 'date_occ':'dt_occurred', 'rpt_dist_no':'district_code', 'crm_cd':'crime_code', 'crm_cd_desc':'crime_description',
                   'premis_cd':'premise_code', 'premis_desc':'premise_description'})
                   

In [58]:
#check new column names
df.columns

Index(['id', 'dt_reported', 'dt_occurred', 'time_occ', 'area', 'area_name',
       'district_code', 'part_1_2', 'crime_code', 'crime_description',
       'vict_age', 'vict_sex', 'vict_descent', 'premise_code',
       'premise_description', 'status', 'status_desc', 'crm_cd_1', 'crm_cd_2',
       'location', 'lat', 'lon', 'mocodes', 'weapon_used_cd', 'weapon_desc',
       'cross_street', 'crm_cd_3', 'crm_cd_4'],
      dtype='object')

In [59]:
#check data types
df.dtypes

id                     object
dt_reported            object
dt_occurred            object
time_occ               object
area                   object
area_name              object
district_code          object
part_1_2               object
crime_code             object
crime_description      object
vict_age               object
vict_sex               object
vict_descent           object
premise_code           object
premise_description    object
status                 object
status_desc            object
crm_cd_1               object
crm_cd_2               object
location               object
lat                    object
lon                    object
mocodes                object
weapon_used_cd         object
weapon_desc            object
cross_street           object
crm_cd_3               object
crm_cd_4               object
dtype: object

# This is where we stopped - to figure out time occur data type. please use copy_df instead of df

In [66]:
copy_df = df[['dt_occurred','time_occ']]
copy_df['dt_occurred'] = pd.to_datetime(copy_df['dt_occurred'], errors='coerce').dt.date
copy_df['time_occ'] = pd.to_datetime(copy_df['time_occ'], format='%H:%M').dt.time
copy_df['dt_occurred_combined'] = pd.to_datetime(copy_df['dt_occurred'] + ' ' + df_copy['time_occ'], errors='coerce')

/var/folders/pt/98lphcwn2w98bfr_79q2qdrw0000gn/T/ipykernel_1821/2661323005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_df['dt_occurred'] = pd.to_datetime(copy_df['dt_occurred'], errors='coerce').dt.date


ValueError: time data "2130" doesn't match format "%H:%M", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [65]:
copy_df.head()

,dt_occurred,time_occ
0,2020-03-01,2130
1,2020-02-08,1800
2,2020-11-04,1700
3,2020-03-10,2037
4,2020-08-17,1200


In [39]:
#change data type of date_time
df['dt_reported'] = pd.to_datetime(df['dt_reported'], errors='coerce')
df['dt_occurred'] = pd.to_datetime(df['dt_occurred'], errors='coerce')
def convert_to_time(t):
    t = str(t).zfill(4)  # Ensure the string has 4 characters, pad with zeroes if needed
    return f'{t[:2]}:{t[2:]}'
df['time_occ'] = df['time_occ'].apply(convert_to_time)
df['time_occ'] = pd.to_datetime(df['time_occ'], errors = 'coerce', format='%H:%M')
df.head()

,id,dt_reported,dt_occurred,time_occ,area,area_name,district_code,part_1_2,crime_code,crime_description,...,crm_cd_2,location,lat,lon,mocodes,weapon_used_cd,weapon_desc,cross_street,crm_cd_3,crm_cd_4
0,190326475,2020-03-01,2020-03-01,NaT,07,Wilshire,0784,1,510,VEHICLE - STOLEN,...,998,1900 S LONGWOOD AV,34.0375,-118.3506,NaN,NaN,NaN,NaN,NaN,NaN
1,200106753,2020-02-09,2020-02-08,NaT,01,Central,0182,1,330,BURGLARY FROM VEHICLE,...,998,1000 S FLOWER ST,34.0444,-118.2628,1822 1402 0344,NaN,NaN,NaN,NaN,NaN
2,200320258,2020-11-11,2020-11-04,NaT,03,Southwest,0356,1,480,BIKE - STOLEN,...,NaN,1400 W 37TH ST,34.021,-118.3002,0344 1251,NaN,NaN,NaN,NaN,NaN
3,200907217,2023-05-10,2020-03-10,NaT,09,Van Nuys,0964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,NaN,14000 RIVERSIDE DR,34.1576,-118.4387,0325 1501,NaN,NaN,NaN,NaN,NaN
4,220614831,2022-08-18,2020-08-17,NaT,06,Hollywood,0666,2,354,THEFT OF IDENTITY,...,NaN,1900 TRANSIENT,34.0944,-118.3277,1822 1501 0930 2004,NaN,NaN,NaN,NaN,NaN


In [41]:
#fill na w/ 0
df = df.fillna(0)

In [44]:
#change data type of integers
#id, area, district_code, part_1_2, crime_code, mocodes, vict_age, premise code, weapon used code, crm_cd_1, crm_cd_2, crm_cd_3, crm_cd_4, lat, lon
df[['id', 'area', 'district_code', 'part_1_2', 'crime_code', 'vict_age', 'premise_code', 'weapon_used_cd', 'crm_cd_1', 'crm_cd_2', 'crm_cd_3', 'crm_cd_4']]= df[['id', 'area', 'district_code', 'part_1_2', 'crime_code', 'vict_age', 'premise_code', 'weapon_used_cd', 'crm_cd_1', 'crm_cd_2', 'crm_cd_3', 'crm_cd_4']].astype('int64')

In [45]:
#change data type of lat lon
df[['lat', 'lon']] = df[['lat', 'lon']].astype('float')

In [46]:
#check data types
df.dtypes

id                              int64
dt_reported            datetime64[ns]
dt_occurred            datetime64[ns]
time_occ                        int64
area                            int64
area_name                      object
district_code                   int64
part_1_2                        int64
crime_code                      int64
crime_description              object
vict_age                        int64
vict_sex                       object
vict_descent                   object
premise_code                    int64
premise_description            object
status                         object
status_desc                    object
crm_cd_1                        int64
crm_cd_2                        int64
location                       object
lat                           float64
lon                           float64
mocodes                        object
weapon_used_cd                  int64
weapon_desc                    object
cross_street                   object
crm_cd_3    

In [47]:
df.head()

,id,dt_reported,dt_occurred,time_occ,area,area_name,district_code,part_1_2,crime_code,crime_description,...,crm_cd_2,location,lat,lon,mocodes,weapon_used_cd,weapon_desc,cross_street,crm_cd_3,crm_cd_4
0,190326475,2020-03-01,2020-03-01,0,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,998,1900 S LONGWOOD AV,34.0375,-118.3506,0,0,0,0,0,0
1,200106753,2020-02-09,2020-02-08,0,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,998,1000 S FLOWER ST,34.0444,-118.2628,1822 1402 0344,0,0,0,0,0
2,200320258,2020-11-11,2020-11-04,0,3,Southwest,356,1,480,BIKE - STOLEN,...,0,1400 W 37TH ST,34.0210,-118.3002,0344 1251,0,0,0,0,0
3,200907217,2023-05-10,2020-03-10,0,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,0,14000 RIVERSIDE DR,34.1576,-118.4387,0325 1501,0,0,0,0,0
4,220614831,2022-08-18,2020-08-17,0,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,0,1900 TRANSIENT,34.0944,-118.3277,1822 1501 0930 2004,0,0,0,0,0
